In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [5]:
ls

'cw1_template 2022(1).ipynb'    Simpsons2022/      Task1test.ipynb
 development-examples.txt.txt   Simpsons2022.zip   Task2.ipynb
 product_reviews/               Task1.ipynb


In [6]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import gensim
from gensim.models import Word2Vec
import os 
import re
import copy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
import string
import torchtext
from torchtext.data import get_tokenizer
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from nltk import cluster
from nltk.stem import WordNetLemmatizer
from random import choices
from nltk.corpus import stopwords, wordnet, words
from nltk.corpus import wordnet
from nltk.cluster import GAAClusterer
import multiprocessing
nltk.download('words')

def pos_tagger (nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:         
    return None

def preprocess(document: str):
  raw = document.read()
 
  #store each lines with lower case 
  raw_lines = [line for line in raw.lower().split("\n") if line !=""]

  # get all the setences  
  reviews = []
  review = []
  for i in raw_lines:
    #review which are divided in tag[t]
    if i == '[t]':
      reviews.append(review)
      review = []
    else:
      # if found ##tag then split and append in to array
      if "##" in i:
        i = i.split("##")
        review.append(i[1])
      else:
        continue
  reviews.append(review)

  #delete the empty array
  if not reviews[0]:
    del reviews[0]

  #tokenize the word in review
  tokens = []
  tokenizer = get_tokenizer('basic_english')
  for review in reviews:
    for sentence in review:
      tokens.append(tokenizer(sentence))
    
  #lemmatization
  lemmatizer=WordNetLemmatizer()

  # pos tagging
  token = []
  for sentence in tokens:
    pos_tagged = nltk.pos_tag(sentence)
    wordnet_tagged = [(x[0], pos_tagger(x[1])) for x in pos_tagged]
  
    # lemmatisation
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
            # print(lemmatized_sentence)   
    token.append(lemmatized_sentence)

  #remove some symbols still exist
  punctuations_symbols = "[]≠−→←“•’´―⁄ .!?#$%^&*(){}~`|><,”'" +  (string.punctuation)
  for sentence in token:
    for items in sentence:
      if items in punctuations_symbols:
        sentence.remove(items)

  #remove stop words    
  stop_words = set(stopwords.words())
  
  tokens =[]
  for sentence in token:
    tokens.append([word for word in sentence if not word in stop_words])

  return tokens


def main():
  path = "product_reviews/product_reviews/"
  filelist = os.listdir(path)

  counters = []
  wordvec = []
  allwords = []

  #reach each txt file and put things seperately 
  #wordlist has the all the word to check top50 words
  #wordvec has the position of  all the review and words
  #counter store each counters for tokenized each documentary
  for x in filelist:
    if x.endswith(".txt"):
      document = open(path+x, "r")
      token = preprocess(document)
      wordvec+= (token)
      wordlist= []
      for i in token:
        wordlist+= i
      allwords +=  wordlist
      counters_oj =  Counter(wordlist)
      counters.append(counters_oj)

  counter_obj = Counter(allwords)
  counter50 = copy.deepcopy(counter_obj)

  dellist = []

  # delete symmetric
  for i in counter_obj:
    if i == i[::-1]:
      dellist.append(i)

  for i in dellist:
    del counter_obj[i]
    del counter50[i]


  #most 50
  most50 = counter50.most_common(n=50)

  print(most50)

  percentages = []
  mean =0
  std = 0
  

  # get term for most 50 and check how many reverse word should be
  terms = []
  rest_most50 = {}
  for word in most50:
    terms.append(word[0])
    half = int(word[1]/2)
    rest = word[1] - half
    rest_most50[word[0]] =rest

  #every document should have that much words
  data = {}
  for i in most50:
    for docu in counters:
      half = int(docu[i[0]]/2)
      rest = docu[i[0]] - half
      if i[0] in data:
        data[i[0]].append(half)
      else:
        data[i[0]] = [half]

      if i[0][::-1] in data:
        data[i[0][::-1]].append(rest)
      else:
        data[i[0][::-1]] = [rest]


  #get the index of each top50 words
  topword_index = {}
  for i in range(len(wordvec)):
    for j in range(len(wordvec[i])):
      if wordvec[i][j] in terms:
        if wordvec[i][j] in topword_index:
          topword_index[wordvec[i][j]].append([i,j])
        else:
          topword_index[wordvec[i][j]] = []
          topword_index[wordvec[i][j]].append([i,j])
    
  # run 10 times of checking percentage of clusterting 
  for i in range(10):
    #deep copy of wordvec
    wordvec_cp = copy.deepcopy(wordvec)
    terms_cp = copy.deepcopy(terms)
    
    #randomly reverse the word in the whole tokens
    for r_word in data:
      if r_word not in terms_cp:
        for num in range(rest_most50[r_word[:: -1]]):
          [pos] = choices(topword_index[r_word[::-1]])
          wordvec_cp[pos[0]][pos[1]] = r_word
  
    # get all terms for target and reversed one
    terms_all = []
    for i in data:
      terms_all.append(i)

    cores = multiprocessing.cpu_count()
    #word2vec model set parameters  window size 10 min_count 5
    w2v  = Word2Vec(min_count=5,
                      window=10,
                      sg =1,
                      size=100,
                      workers=cores-1)
    #Build word2vec vocabulary
    w2v.build_vocab(wordvec_cp, progress_per=10000)
    #Traing word2vec copy every time
    w2v.train(wordvec_cp, total_examples=w2v.corpus_count, epochs=20, report_delay=1)
    # get vector of vocabulary
    w2v_vectors = w2v.wv.vectors
    # get indicies of vocabulary
    w2v_indices = {word: w2v.wv.vocab[word].index for word in w2v.wv.vocab}

    # getting feature vector for each terms
    feature_vecs = []
    for i in terms_all:
      feature_vec = w2v_vectors[w2v_indices[i]]
      feature_vecs.append(feature_vec)

    #use GAACcluster
    clusterer = GAAClusterer(50)
    labels = clusterer.cluster(feature_vecs, True)
    p = 0
    
    #checking labels every time
    for i in range(50):
      if labels[2*i] == labels[2*i +1]:
        p+=1
    p = p /50 *100
    print("correct pair",p,"percentage")
    percentages.append(p)
  
  #print out mean and standar deviation of percentages
  percentages=  np.array(percentages)
  mean = np.mean(percentages)
  print("mean:",mean)
  std =  np.std(percentages)
  print("standard deviation:",std)

result = main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


[('ipod', 309), ('phone', 269), ('router', 264), ('camera', 248), ('work', 237), ('player', 237), ('great', 195), ('buy', 187), ('problem', 179), ('time', 179), ('battery', 174), ('diaper', 171), ('product', 159), ('make', 156), ('easy', 147), ('computer', 143), ('feature', 134), ('quality', 125), ('give', 119), ('micro', 119), ('find', 115), ('creative', 112), ('software', 108), ('picture', 103), ('sound', 99), ('music', 96), ('bag', 96), ('review', 94), ('purchase', 93), ('mp3', 93), ('song', 93), ('bit', 89), ('lot', 85), ('norton', 82), ('month', 81), ('year', 81), ('run', 81), ('small', 79), ('card', 78), ('file', 77), ('cd', 76), ('wireless', 76), ('service', 76), ('linksys', 76), ('apple', 75), ('program', 75), ('support', 70), ('champ', 70), ('day', 68), ('download', 67)]
correct pair 78.0 percentage
correct pair 74.0 percentage
correct pair 70.0 percentage
correct pair 72.0 percentage
correct pair 76.0 percentage
correct pair 74.0 percentage
correct pair 74.0 percentage
correc